In [1]:
import struct, bluetooth

In [2]:
# Check if our unit can be detected (for validating bluetooth.connect())

devs = bluetooth.discover_devices()
print(devs)

['00:06:66:D7:C6:F2']


In [3]:
# Connect to GSR unit

gsr_address = '00:06:66:D7:C6:F2'
shimmer_gsr = bluetooth.BluetoothSocket(bluetooth.RFCOMM)
shimmer_gsr.connect((gsr_address, 1))

In [4]:
# Utilities

def pack_char(val):
    return struct.pack('B', val)

# Packet type variables
DATA_PACKET            = pack_char(0x00)
INQUIRY_COMMAND        = pack_char(0x01)
INQUIRY_RESPONSE       = pack_char(0x02)
SET_SENSORS_COMMAND    = pack_char(0x08)
ACKNOWLEDGE            = pack_char(0xFF)

# Sensor variables
SENSOR_GSR             = pack_char(0x04)


def wait_ACK():
    while True:
        response = shimmer_gsr.recv(1)
        print('wait for ack', response)
        if response == ACKNOWLEDGE:
            print('Shimmer: ACK')
            return

In [5]:
# Enable GSR readings
# https://raw.githubusercontent.com/ShimmerResearch/shimmer3/LogAndStream_v0.11.0/LogAndStream/shimmer_btsd.h

set_gsr_command = SET_SENSORS_COMMAND + SENSOR_GSR + 2*DATA_PACKET

# SET_SENSORS_COMMAND = 0x08
# SENSOR_GSR = 0x04
# set_gsr_command = struct.pack('BBBB', SET_SENSORS_COMMAND, SENSOR_GSR, 0x00, 0x00) 

shimmer_gsr.send(set_gsr_command)
wait_ACK()

wait for ack b'\xff'
Shimmer: ACK


In [6]:
# Shitty way of finding our GSR channel through inquiry command

shimmer_gsr.send(INQUIRY_COMMAND)
wait_ACK()


def read_inqury_response():
    counter = 0
    num_channels = 0
    packet_begin = False
    packet_end = 999 # Arbitrary large enough number
    response = b''
 
    while counter <= packet_end:
        data = shimmer_gsr.recv(1)
        if data == INQUIRY_RESPONSE:
            packet_begin = True
        if packet_begin:
            if counter == 7:
                packet_end = 8 + data[0]
            response += data
            counter += 1
    return response
            
    
def print_inquiry_response(response):
    print('Full response', list(response))
    print('Response length', len(response))
    print('Packet type', hex(response[0]))
    print('Sample rate', hex(response[1]))
    print('Sample rate', hex(response[2]))
    print('Config byte', hex(response[3]))
    print('Config byte', hex(response[4]))
    print('Config byte', hex(response[5]))
    print('Config byte', hex(response[6]))
    print('Number of channels', hex(response[7]))
    print('Buffer size', hex(response[8]))
    for i in range(response[7]):
        print('Channel %s' % (i + 1), hex(response[9 + i]))
        
        
inquiry_response = read_inqury_response()
print_inquiry_response(inquiry_response)

wait for ack b'\xff'
Shimmer: ACK
Full response [2, 0, 1, 81, 255, 12, 9, 1, 1, 28]
Response length 10
Packet type 0x2
Sample rate 0x0
Sample rate 0x1
Config byte 0x51
Config byte 0xff
Config byte 0xc
Config byte 0x9
Number of channels 0x1
Buffer size 0x1
Channel 1 0x1c


In [ ]:
# # Check for packet begin outside func
# def read_stream_packet(sock, packet_channel_size, overflow_data=b''):
#     packet = overflow_data
#     for i in range(3 + packet_size): # Add timestamp to size
#         packet += sock.recv(1)
#     return packet


# def read_stream_cmd_response():
    

# def read_stream(sock, shimmer_config):
#     while True:
#         data = sock.recv(1)
#         if data == DATA_PACKET:
#             packet = read_stream_packet(sock, shimmer_config[7])

In [7]:
shimmer_gsr.close()